# Amazon Bedrock Model Catalog - Demonstration Notebook

This notebook demonstrates the BedrockModelCatalog class for accessing Amazon Bedrock model information through AWS APIs.

## Features Demonstrated

1. **Basic Model Data Retrieval**: Access the latest model information from AWS APIs
2. **Data Exploration**: Analyze model distribution by provider and region
3. **Filtering and Querying**: Find specific models based on criteria
4. **Model Availability Checks**: Verify model availability in specific regions
5. **Error Handling**: Demonstrate robust error handling
6. **Advanced Configuration**: Custom settings and caching behavior

## Prerequisites

Make sure you have the bestehorn-llmmanager package installed:
```bash
pip install bestehorn-llmmanager
```

## 1. Setup and Imports

In [ ]:
# Standard library imports
import logging
from pathlib import Path
from collections import Counter, defaultdict

# Third-party imports for data analysis and visualization
try:
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    VISUALIZATION_AVAILABLE = True
except ImportError:
    print("⚠️ Visualization libraries not available.")
    print("   Install with: pip install pandas matplotlib seaborn")
    VISUALIZATION_AVAILABLE = False

# Import BedrockModelCatalog
from bestehorn_llmmanager.bedrock.catalog import BedrockModelCatalog, CacheMode

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

print("✅ Imports successful!")
print("💡 BedrockModelCatalog automatically fetches fresh data from AWS APIs")

## 2. Basic Usage - Initialize BedrockModelCatalog

In [ ]:
# Initialize BedrockModelCatalog with force_refresh for demonstration
catalog = BedrockModelCatalog(
    force_refresh=True,  # Always fetch fresh data for demo
    timeout=60,          # Longer timeout for reliability
    fallback_to_bundled=True  # Fallback if API fails
)

print(f"BedrockModelCatalog configuration:")
print(f"  Cache mode: {catalog.cache_mode.value}")
print(f"  Cache file: {catalog.cache_file_path}")
print(f"  Catalog loaded: {catalog.is_catalog_loaded}")

In [ ]:
# Get catalog metadata to see what data we have
print("📊 Retrieving catalog metadata...\n")

try:
    metadata = catalog.get_catalog_metadata()
    
    print(f"✅ Successfully retrieved model data!")
    print(f"📊 Source: {metadata.source.value}")
    print(f"🕐 Retrieved at: {metadata.retrieval_timestamp}")
    print(f"🌍 Regions queried: {len(metadata.api_regions_queried)}")
    
    # Get all models to count them
    all_models = catalog.list_models()
    print(f"📋 Total models: {len(all_models)}")
    
except Exception as e:
    print(f"❌ Error retrieving model data: {e}")
    print("💡 Check your AWS credentials and network connection")

## 3. Exploring the Model Catalog

In [ ]:
# Display first few models to understand the data structure
print("📋 Sample of available models:\n")

all_models = catalog.list_models()
sample_models = all_models[:5]  # First 5 models

for model_info in sample_models:
    print(f"🔹 {model_info.friendly_name}")
    print(f"   Provider: {model_info.provider}")
    print(f"   Model ID: {model_info.model_id}")
    regions = model_info.get_supported_regions()
    print(f"   Regions: {', '.join(regions[:3])}{'...' if len(regions) > 3 else ''}")
    print(f"   Streaming: {'✅' if model_info.supports_streaming else '❌'}")
    print()

## 4. Provider Analysis

In [ ]:
# Analyze models by provider
all_models = catalog.list_models()
provider_counts = Counter()
provider_streaming = defaultdict(list)

for model_info in all_models:
    provider_counts[model_info.provider] += 1
    provider_streaming[model_info.provider].append(model_info.supports_streaming)

print("📈 Models by Provider:\n")
for provider, count in provider_counts.most_common():
    streaming_count = sum(provider_streaming[provider])
    streaming_pct = (streaming_count / count) * 100
    print(f"🏢 {provider}: {count} models ({streaming_count} support streaming - {streaming_pct:.1f}%)")

# Demonstrate provider filtering using list_models()
print("\n🔍 Amazon models:")
amazon_models = catalog.list_models(provider="Amazon")
for model_info in amazon_models[:3]:
    print(f"   • {model_info.friendly_name}")
if len(amazon_models) > 3:
    print(f"   ... and {len(amazon_models) - 3} more")

## 5. Regional Analysis

In [ ]:
# Analyze models by AWS region
all_models = catalog.list_models()
region_counts = Counter()

for model_info in all_models:
    for region in model_info.get_supported_regions():
        region_counts[region] += 1

print("🌍 Top 10 Regions by Model Availability:\n")
for region, count in region_counts.most_common(10):
    print(f"📍 {region}: {count} models")

# Demonstrate region filtering using list_models()
print("\n🔍 Models available in us-east-1:")
us_east_models = catalog.list_models(region="us-east-1")
print(f"   Total: {len(us_east_models)} models")

# Show a few examples
for model_info in us_east_models[:5]:
    print(f"   • {model_info.friendly_name}")
if len(us_east_models) > 5:
    print(f"   ... and {len(us_east_models) - 5} more")

## 6. Modality Analysis

In [ ]:
# Analyze input and output modalities
all_models = catalog.list_models()
input_modalities = Counter()
output_modalities = Counter()

for model_info in all_models:
    # Get access info for first available region to check modalities
    regions = model_info.get_supported_regions()
    if regions:
        access_info = model_info.get_access_info_for_region(region=regions[0])
        if access_info:
            for modality in access_info.input_modalities:
                input_modalities[modality] += 1
            for modality in access_info.output_modalities:
                output_modalities[modality] += 1

print("🎯 Input Modalities:\n")
for modality, count in input_modalities.most_common():
    print(f"   📥 {modality}: {count} models")

print("\n🎯 Output Modalities:\n")
for modality, count in output_modalities.most_common():
    print(f"   📤 {modality}: {count} models")

# Find multimodal models
multimodal_input = []
multimodal_output = []
for model_info in all_models:
    regions = model_info.get_supported_regions()
    if regions:
        access_info = model_info.get_access_info_for_region(region=regions[0])
        if access_info:
            if len(access_info.input_modalities) > 1:
                multimodal_input.append(model_info.friendly_name)
            if len(access_info.output_modalities) > 1:
                multimodal_output.append(model_info.friendly_name)

print(f"\n🔀 Multimodal Models:")
print(f"   Multiple inputs: {len(multimodal_input)} models")
print(f"   Multiple outputs: {len(multimodal_output)} models")

## 7. Streaming Support Analysis

In [ ]:
# Analyze streaming support using list_models()
all_models = catalog.list_models()
streaming_models = catalog.list_models(streaming_only=True)
total_models = len(all_models)
streaming_percentage = (len(streaming_models) / total_models) * 100

print(f"🚀 Streaming Support Analysis:\n")
print(f"   Total models: {total_models}")
print(f"   Streaming supported: {len(streaming_models)} ({streaming_percentage:.1f}%)")
print(f"   No streaming: {total_models - len(streaming_models)} ({100 - streaming_percentage:.1f}%)")

print("\n🔍 Sample streaming-enabled models:")
for model_info in streaming_models[:5]:
    print(f"   • {model_info.friendly_name} ({model_info.provider})")

if len(streaming_models) > 5:
    print(f"   ... and {len(streaming_models) - 5} more")

## 8. Model Availability Checks

In [ ]:
# Demonstrate model availability checking with is_model_available()
print("🔍 Model Availability Checks:\n")

# Test some common models
test_cases = [
    ("Claude 3 Haiku", "us-east-1"),
    ("Claude 3 Sonnet", "us-west-2"),
    ("Titan Text G1 - Express", "eu-west-1"),
    ("NonExistentModel", "us-east-1"),
]

for model_name, region in test_cases:
    is_available = catalog.is_model_available(model_name=model_name, region=region)
    status = "✅ Available" if is_available else "❌ Not available"
    print(f"{status}: {model_name} in {region}")

# Get detailed model info using get_model_info()
print("\n📋 Detailed Model Information:\n")
model_info = catalog.get_model_info(model_name="Claude 3 Haiku", region="us-east-1")
if model_info:
    print(f"🤖 Model: Claude 3 Haiku")
    print(f"   Model ID: {model_info.model_id}")
    print(f"   Inference Profile: {model_info.inference_profile_id or 'N/A'}")
    print(f"   Access Method: {model_info.access_method.value}")
    print(f"   Streaming: {'✅' if model_info.supports_streaming else '❌'}")
    print(f"   Input Modalities: {', '.join(model_info.input_modalities)}")
    print(f"   Output Modalities: {', '.join(model_info.output_modalities)}")
else:
    print("❌ Model not found")

## 9. Visualization (if libraries available)

In [ ]:
# Create visualizations if pandas/matplotlib are available
if VISUALIZATION_AVAILABLE and 'catalog' in locals():
    print("📊 Creating visualizations...\n")
    
    # Prepare data for visualization
    model_data = []
    for name, info in catalog.models.items():
        model_data.append({
            'name': name,
            'provider': info.provider,
            'streaming': info.streaming_supported,
            'num_regions': len(info.regions_supported),
            'num_input_modalities': len(info.input_modalities),
            'num_output_modalities': len(info.output_modalities)
        })
    
    df = pd.DataFrame(model_data)
    
    # Set up the plotting style
    plt.style.use('default')
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Amazon Bedrock Models Analysis', fontsize=16, fontweight='bold')
    
    # Provider distribution
    provider_counts = df['provider'].value_counts()
    axes[0, 0].pie(provider_counts.values, labels=provider_counts.index, autopct='%1.1f%%')
    axes[0, 0].set_title('Models by Provider')
    
    # Streaming support by provider
    streaming_by_provider = df.groupby(['provider', 'streaming']).size().unstack(fill_value=0)
    streaming_by_provider.plot(kind='bar', stacked=True, ax=axes[0, 1], 
                              color=['lightcoral', 'lightgreen'])
    axes[0, 1].set_title('Streaming Support by Provider')
    axes[0, 1].set_xlabel('Provider')
    axes[0, 1].set_ylabel('Number of Models')
    axes[0, 1].legend(['No Streaming', 'Streaming Supported'])
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    # Region availability distribution
    axes[1, 0].hist(df['num_regions'], bins=10, edgecolor='black', alpha=0.7)
    axes[1, 0].set_title('Distribution of Regional Availability')
    axes[1, 0].set_xlabel('Number of Regions')
    axes[1, 0].set_ylabel('Number of Models')
    
    # Modality complexity
    axes[1, 1].scatter(df['num_input_modalities'], df['num_output_modalities'], 
                      alpha=0.6, s=60)
    axes[1, 1].set_title('Input vs Output Modalities')
    axes[1, 1].set_xlabel('Number of Input Modalities')
    axes[1, 1].set_ylabel('Number of Output Modalities')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Visualizations complete!")
    
else:
    print("📊 Visualization libraries not available or no data to visualize")

## 10. Error Handling and Troubleshooting

In [ ]:
# Demonstrate error handling scenarios
print("🧪 Testing error handling scenarios...\n")

# Test 1: Non-existent model
print("Test 1: Non-existent model")
model_info = catalog.get_model_info(model_name="NonExistentModel", region="us-east-1")
if model_info is None:
    print("✅ Correctly returned None for non-existent model\n")

# Test 2: Invalid region
print("Test 2: Model in invalid region")
is_available = catalog.is_model_available(model_name="Claude 3 Haiku", region="invalid-region")
if not is_available:
    print("✅ Correctly returned False for invalid region\n")

# Test 3: Empty filter results
print("Test 3: Filter with no matches")
no_models = catalog.list_models(provider="NonExistentProvider")
if len(no_models) == 0:
    print("✅ Correctly returned empty list for non-existent provider\n")

print("🎯 Error handling tests complete!")

# Troubleshooting tips
print("\n💡 Troubleshooting Tips:")
print("   • Import errors: Ensure bestehorn-llmmanager is installed")
print("   • API timeouts: Increase timeout parameter or check network")
print("   • No models found: Check AWS credentials and permissions")
print("   • Cache issues: Use force_refresh=True or clear_cache()")

## 11. Advanced Configuration Example

In [ ]:
# Demonstrate advanced configuration options
print("⚙️ Advanced Configuration Examples\n")

# Example 1: Memory-only caching (Lambda-friendly)
print("Example 1: Memory-only caching")
memory_catalog = BedrockModelCatalog(
    cache_mode=CacheMode.MEMORY,
    force_refresh=False,
    fallback_to_bundled=True
)
print(f"  Cache mode: {memory_catalog.cache_mode.value}")
print(f"  Cache file: {memory_catalog.cache_file_path}")

# Example 2: No caching (always fresh)
print("\nExample 2: No caching (always fresh)")
no_cache_catalog = BedrockModelCatalog(
    cache_mode=CacheMode.NONE,
    fallback_to_bundled=True
)
print(f"  Cache mode: {no_cache_catalog.cache_mode.value}")
print(f"  Always fetches fresh data from APIs")

# Example 3: Custom cache directory
print("\nExample 3: Custom cache directory")
custom_catalog = BedrockModelCatalog(
    cache_mode=CacheMode.FILE,
    cache_directory=Path("./demo_cache"),
    cache_max_age_hours=12.0,
    force_refresh=False
)
print(f"  Cache mode: {custom_catalog.cache_mode.value}")
print(f"  Cache directory: {custom_catalog.cache_file_path}")
print(f"  Max age: 12 hours")

print("\n📋 Catalog representation:")
print(f"  Current catalog: {catalog}")

## 12. Performance and Usage Summary

In [ ]:
# Summarize what we've learned
print("📈 Performance and Usage Summary\n")

metadata = catalog.get_catalog_metadata()
all_models = catalog.list_models()

print(f"✅ Successfully processed {len(all_models)} models")
print(f"📅 Data retrieved: {metadata.retrieval_timestamp}")
print(f"📊 Data source: {metadata.source.value}")
print(f"🌍 Regions queried: {len(metadata.api_regions_queried)}")

if metadata.cache_file_path:
    cache_size = metadata.cache_file_path.stat().st_size
    print(f"💾 Cache file size: {cache_size:,} bytes ({cache_size/1024:.1f} KB)")

print("\n🎯 Key Takeaways:")
print("   • BedrockModelCatalog provides unified access to model information")
print("   • Supports filtering by provider, region, and streaming capability")
print("   • Uses is_model_available() for availability checks")
print("   • Uses get_model_info() for detailed model information")
print("   • Uses list_models() for filtering and discovery")
print("   • Handles errors gracefully with informative responses")
print("   • Supports multiple caching strategies (FILE, MEMORY, NONE)")
print("   • Automatically fetches from AWS APIs with bundled fallback")

print("\n🚀 Ready for production use!")

## 13. Next Steps and Integration Examples

Here are some ways you might integrate BedrockModelCatalog into your applications:

### Automated Model Discovery
```python
# Check model availability before making API calls
def get_available_model(preferred_models, region):
    catalog = BedrockModelCatalog()
    for model_name in preferred_models:
        if catalog.is_model_available(model_name=model_name, region=region):
            return catalog.get_model_info(model_name=model_name, region=region)
    return None
```

### Region-Specific Model Selection
```python
# Find best model for specific region and requirements
def find_suitable_models(region, streaming_required=False, provider_preference=None):
    catalog = BedrockModelCatalog()
    return catalog.list_models(
        region=region,
        streaming_only=streaming_required,
        provider=provider_preference
    )
```

### Cost Optimization
```python
# Analyze model availability across regions for cost optimization
def analyze_regional_options(model_name):
    catalog = BedrockModelCatalog()
    all_models = catalog.list_models()
    
    for model_info in all_models:
        if model_info.friendly_name == model_name:
            regions = model_info.get_supported_regions()
            return {
                'model_id': model_info.model_id,
                'regions': regions,
                'streaming': model_info.supports_streaming,
                'provider': model_info.provider
            }
    return None
```

## Documentation

For complete documentation, see:
- `docs/forLLMConsumption.md` - Complete API documentation
- `README.md` - BedrockModelCatalog overview
- Source code in `src/bestehorn_llmmanager/bedrock/catalog/` - Fully documented implementation
- This notebook - Interactive examples and demonstrations

## Migration from Legacy Managers

If you're migrating from the old ModelManager:
- Replace `ModelManager()` with `BedrockModelCatalog()`
- Replace `refresh_model_data()` with automatic initialization
- Replace `get_models_by_provider()` with `list_models(provider=...)`
- Replace `get_models_by_region()` with `list_models(region=...)`
- Replace `get_streaming_models()` with `list_models(streaming_only=True)`
- Use `is_model_available()` for availability checks
- Use `get_model_info()` for detailed model information